# Decision Trees for Regression

Compared to last week, this is a very simple lab <span style="font-size:20pt;">😃</span>

You will implement the [Classification and Regression Tree (CART)](https://en.wikipedia.org/wiki/Predictive_analytics#Classification_and_regression_trees_.28CART.29) algorithm from scratch.

The lab is broken down into the following pieces:

* Regression Criterion
* Creating Splits
* Buiding a Tree
* Making a prediction

## Exercise 1 -- Download and load the dataset

This time, we will be using the banknote authentication data set, which is available to download from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/banknote+authentication)

* Download the file
* Read it and make a scikit-learn style dataset from it
* Make a 70/30 train test split

**TIP:** Pandas can readl URLs directly (But it's still nice to have the dataset locally)

In [1]:
# your code here

In [2]:
data

,variance,skewness,curtosis,entropy,class
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


Make a dataset where the features are

* `variance`
* `skewness`
* `curtosis`

and the target variable is

* `entropy`

For this lab, simply ignore the `class` column

In [3]:
# your code here

## Exercise 2 -- Optimization Criterion

For regression, a simple criterion to optimize is to minimize the sum of squared errors for a given region. This is, for all datapoints in a region with size, we minimize:

$$\sum_{i=1}^N(y_i - \hat{y})^2$$

where $N$ is the number of datapoits in the region and $\hat{y}$ is the mean value of the region for the target variable. 

Implement such a function using the description below.

Please, don't use an existing implementation, refer to the [book](https://www.statlearning.com/s/ISLRSeventhPrinting.pdf), and if you need help, as questions!

In [4]:
def regression_criterion(region):
    """
    Implements the sum of squared error criterion in a region
    
    Parameters
    ----------
    region : ndarray
        Array of shape (N,) containing the values of the target values 
        for N datapoints in the training set.
    
    Returns
    -------
    float
        The sum of squared error
        
    Note
    ----
    The error for an empty region should be infinity
    This avoids creating empty regions
    """
    # your code here

In [5]:
# test your code
rng = np.random.default_rng(0)
print(regression_criterion(rng.random(size=40)))
print(regression_criterion(np.ones(10)))
print(regression_criterion(np.zeros(10)))
print(regression_criterion(np.array([])))

2.7733391199176196e-30
0.0
0.0
inf


## Exercise 3 -- Make a split

In [6]:
def split_region(region, feature_index, tau):
    """
    Given a region, splits it based on the feature indicated by
    `feature_index`, the region will be split in two, where
    one side will contain all points with the feature with values 
    lower than `tau`, and the other split will contain the 
    remaining datapoints.
    
    Parameters
    ----------
    region : array of size (n_samples, n_features)
        a partition of the dataset (or the full dataset) to be split
    feature_index : int
        the index of the feature (column of the region array) used to make this partition
    tau : float
        The threshold used to make this partition
        
    Return
    ------
    low_partition : array
        indices of the datapoints in `region` where feature < `tau`
    high_partition : array
        indices of the datapoints in `region` where feature >= `tau` 
    """
    # your code here

In [7]:
r, l = split_region(X_train, 1, 0)
print(r.shape, l.shape)

(324,) (636,)


## Exercise 4 -- Find the best split

The strategy is quite simple (as well as inefficient), but it helps to reinforce the concepts.
We are going to use a greedy, exhaustive algorithm to select splits, selecting the `feature_index` and the `tau` that minimizes the Regression Criterion

In [8]:
def get_split(X, y):
    """
    Given a dataset (full or partial), splits it on the feature of that minimizes the sum of squared error
    
    Parameters
    ----------
    X : array (n_samples, n_features)
        features 
    y : array (n_samples, )
        labels
    
    Returns
    -------
    decision : dictionary
        keys are:
        * 'feature_index' -> an integer that indicates the feature (column) of `X` on which the data is split
        * 'tau' -> the threshold used to make the split
        * 'low_region' -> array of indices where the `feature_index`th feature of X is lower than `tau`
        * 'high_region' -> indices not in `low_region`
    """
    # your code here

In [9]:
get_split(X_train[:5, :], y_train[:5])

{'feature_index': 0,
 'tau': 0.6818,
 'low_region': array([0, 1, 2, 4]),
 'high_region': array([3])}

## Exercise 5 -- Recursive Splitting

The test above is an example on how to find the root node of our decision tree. The algorithm now is a greedy search until we reach a stop criterion.

The trivial stopping criterion is to recursively grow the tree until each split contains a single point (perfect node purity). If we go that far, it normally means we are overfitting.

You will implement these criteria to stop the growth:

* A node is a leaf if:
    * It has less than `min_samples` datapoints
    * It is at the `max_depth` level from the root (each split creates a new level)
    * The criterion is `0`



In [10]:
def recursive_growth(node, min_samples, max_depth, current_depth, X, y):
    """
    Recursively grows a decision tree.
    
    Parameters
    ----------
    node : dictionary
        If the node is terminal, it contains only the "value" key, which determines the value to be used as a prediction.
        If the node is not terminal, the dictionary has the structure defined by `get_split`
    min_samples : int
        parameter for stopping criterion if a node has <= min_samples datapoints
    max_depth : int
        parameter for stopping criterion if a node belongs to this depth
    depth : int
        current distance from the root
    X : array (n_samples, n_features)
        features (full dataset)
    y : array (n_samples, )
        labels (full dataset)
    
    Notes
    -----
    To create a terminal node, a dictionary is created with a single "value" key, with a value that
    is the mean of the target variable
    
    'left' and 'right' keys are added to non-terminal nodes, which contain (possibly terminal) nodes 
    from higher levels of the tree:
    'left' corresponds to the 'low_region' key, and 'right' to the 'high_region' key
    """
    # your code here

In [11]:
k = 20
test_root = get_split(X_train[:k, :], y_train[:k])
recursive_growth(test_root, 5, 3, 1, X_train[:k, :], y_train[:k])

In [13]:
def print_tree(node, depth):
    if 'value' in node.keys():
        print('.  '*(depth-1), f"[{node['value']}]")
    else:
        print('.  '*depth, f'X_{node["feature_index"]} < {node["tau"]}')
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)


In [14]:
print_tree(test_root, 0)

 X_2 < 0.91995
.   X_0 < -2.0754
.   [-2.268955]
.  .   X_0 < -0.90784
.  .   [-2.9952533333333338]
.  .   [-2.9952533333333338]
.   X_1 < -6.8804
.   [-3.379095]
.  .   X_2 < 0.37158
.  .   [-1.899805]
.  .   [-1.899805]


# Exercise 6 -- Make a Prediction
Use the a node to predict the class of a compatible dataset

In [15]:
def predict_sample(node, sample):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    sample : array of size (n_features,)
        a sample datapoint
    """
    # your code here
        
def predict(node, X):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    X : array of size (n_samples, n_features)
        n_samples predictions will be made
    """
    # your code here

Use the following node to test different parameters for the recursive growth. 

* What happens if you let the tree grow very deep?
* What happens if the minimum number of samples is too big? What happens if it is 1?

In [16]:
from sklearn.metrics import mean_squared_error
root = get_split(X_train, y_train)
recursive_growth(root, 20, 6, 1, X_train, y_train)
train_mse = mean_squared_error(y_train, predict(root, X_train))
test_mse = mean_squared_error(y_test, predict(root, X_test))

print(f'Train MSE : {train_mse}')
print(f'Test MSE : {test_mse}')

Train MSE : 4.582477460507463
Test MSE : 4.704711300439497
